In [1]:
import numpy as np
import csv

from collections import defaultdict

In [2]:
text_file = open("Model2_0.02_Tags.txt", "r")
textFileTags = text_file.readlines()

In [3]:
text_file.close()

In [4]:
modelTags = []
for x in textFileTags:
#     m = x[:len(x)-2]
    m = x.rstrip('\n').split(',')
    modelTags.append(m[0])

In [5]:
# modelTags
len(textFileTags)

11534

In [6]:
words = [x for x in modelTags]

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)
len(wordId)

11534

In [7]:
def parseCSVData(file,data,maxCount):
    rowIndex = 0
    rowCount = 0
    with open(file, 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        for row in spamreader:
            if rowCount == maxCount:
                break
            rowCount+=1
            if rowIndex !=0:
                element = {}
                fieldnames = ("PostId","PostCreationDate","OwnerUserId","OwnerCreationDate","ReputationAtPostCreation","OwnerUndeletedAnswerCountAtPostTime","Title","BodyMarkdown","Tag1","Tag2","Tag3","Tag4","Tag5","PostClosedDate","OpenStatus")
                index = 0
                for i in row:
                    element[fieldnames[index]] = i
                    if index == 0 or index == 5 or index == 4:   
                        element[fieldnames[index]] = int(i)
                    index +=1
                data.append(element)
            else:
                rowIndex = 1
    return

In [8]:
leData1 = []
max = 150001
parseCSVData('output_Final1.csv',leData1,max)

In [12]:
def feature(datum):
    feat  = [0]*len(words)
    if datum['Tag1']!='':
        if datum['Tag1'] in wordId:
            feat[wordId[datum['Tag1']]] = 1
    if datum['Tag2']!='':
         if datum['Tag2'] in wordId:
            feat[wordId[datum['Tag2']]] = 1
    if datum['Tag3']!='':
         if datum['Tag3'] in wordId:
            feat[wordId[datum['Tag3']]] = 1
    if datum['Tag4']!='':
         if datum['Tag4'] in wordId:
            feat[wordId[datum['Tag4']]] = 1
    if datum['Tag5']!='':
         if datum['Tag5'] in wordId:
            feat[wordId[datum['Tag5']]] = 1
    
    feat.append(datum['ReputationAtPostCreation'])
    feat.append(datum['OwnerUndeletedAnswerCountAtPostTime'])
    
    return feat

In [13]:
def result(datum):
    if datum['OpenStatus']!='open':
        return 1
    
    return 0

In [14]:
X = [feature(d) for d in leData1]


In [15]:
y = [result(d) for d in leData1]

len(leData1), len(X), len(y)

(150000, 150000, 150000)

In [16]:
leData1 = ''
len(leData1)
len(leData1), len(X), len(y)

(0, 150000, 150000)

In [21]:
modelTags = 0

In [20]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()

In [22]:
clf.fit(X,y)
print 'done'

done


In [23]:
predictions = clf.predict(X)
X = 0

In [24]:
def classificationVals(predictions,y):
    index = 0
    truePositive = 0
    trueNegative = 0
    falsePositive = 0
    falseNegative = 0
    for x in predictions:
        if y[index] == x:
            if x == 1:
                truePositive+=1
            else:
                trueNegative+=1
        else:
            if y[index] == 1 and x == 0:
                falseNegative+=1
            else:
                falsePositive +=1
        index+=1
    return truePositive,trueNegative,falsePositive,falseNegative

In [25]:
clasVal = classificationVals(predictions,y)
clasVal

(442, 145644, 201, 3713)

In [26]:
def ClassificationAccuracy(TP,TN,FP,FN):
    return (1.0*(TP+TN)/(TN + TP + FP +FN))

In [27]:
ClassificationAccuracy(clasVal[0],clasVal[1],clasVal[2],clasVal[3])

0.9739066666666667

In [33]:
testData = []
max = 200001
parseCSVData('output_Final3.csv',testData,max)
testData = testData[150000:200000]


In [34]:
len(testData)

50000

In [35]:
X1 = [feature(d) for d in testData]
y = [result(d) for d in testData]

In [36]:
Testpredictions = clf.predict(X1)


In [37]:
clasVal = classificationVals(Testpredictions,y)
clasVal

(10, 48719, 4, 1267)

In [38]:
ClassificationAccuracy(clasVal[0],clasVal[1],clasVal[2],clasVal[3])


0.97458